In [ ]:
# %%
# Improved DCGAN Implementation with Checkpoint Resumption for Color Images (RGB)

import os
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from PIL import Image
import glob
import random
import pickle
from tqdm import tqdm
from PIL import ImageEnhance

# %%

# Constants - Adjusted for color images
MAX_IMAGES_PER_CLASS = 10000
RESULT_SAVE_DIR = "Vivian_128x128_results"
LATENT_DIM = 512
IMAGE_SIZE = 128
BATCH_SIZE = 8
EPOCHS_PER_CLASS = 400
DATASET_PATH = "vivian_nsfw_sfw_cropped_combined_faces"
TOLERANCE = 89

# %%
class SingleClassDataGenerator(keras.utils.Sequence):
    '''Improved data generator with better augmentation for color images'''
    
    def __init__(self, dataset_path, batch_size=32, max_images=1000, shuffle=True):
        self.dataset_path = dataset_path
        self.batch_size = batch_size
        self.max_images = max_images
        self.shuffle = shuffle
        
        print(f"Loading images from: {dataset_path}")
        self._build_file_list()
        
        if self.shuffle:
            random.shuffle(self.file_list)
        self.indexes = np.arange(len(self.file_list))
        
    def _build_file_list(self):
        """Build file list for images"""
        if not os.path.exists(self.dataset_path):
            raise ValueError(f"Directory not found: {self.dataset_path}")
        
        # Support multiple image formats
        image_files = []
        for ext in ['*.png', '*.jpg', '*.jpeg', '*.PNG', '*.JPG', '*.JPEG']:
            image_files.extend(glob.glob(os.path.join(self.dataset_path, ext)))
        
        if len(image_files) > self.max_images:
            image_files = random.sample(image_files, self.max_images)
            
        self.file_list = image_files
        print(f"Found {len(self.file_list)} images")
        
    def __len__(self):
        return len(self.file_list) // self.batch_size
        
    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_files = [self.file_list[i] for i in batch_indexes]
        
        X = self._generate_batch(batch_files)
        return X
        
    def _generate_batch(self, batch_files):
        actual_batch_size = len(batch_files)
        X = np.zeros((actual_batch_size, IMAGE_SIZE, IMAGE_SIZE, 3))
    
        for i, original_img_path in enumerate(batch_files):
            
            img_path_to_process = original_img_path
            load_successful = False
            max_attempts = len(batch_files) * 2 
            attempts = 0
    
            while not load_successful and attempts < max_attempts:
                attempts += 1
                try:
                    img = Image.open(img_path_to_process).convert('RGB')
                    w, h = img.size
                    
                    if abs(w - IMAGE_SIZE) <= TOLERANCE and abs(h - IMAGE_SIZE) <= TOLERANCE:
                        img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
                    else:
                        short_side = min(w, h)
                        left = (w - short_side) // 2
                        top = (h - short_side) // 2
                        right = left + short_side
                        bottom = top + short_side
                        img = img.crop((left, top, right, bottom)).resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
    
                    if random.random() < 0.3:
                        img = img.transpose(Image.FLIP_LEFT_RIGHT)
                    if random.random() < 0.3:
                        enhancer = ImageEnhance.Brightness(img)
                        img = enhancer.enhance(random.uniform(0.9, 1.1))
                    
                    if random.random() < 0.3:
                        enhancer = ImageEnhance.Contrast(img)
                        img = enhancer.enhance(random.uniform(0.9, 1.1))
    
                    img_array = np.array(img, dtype=np.float32) / 255.0
                    img_array = (img_array - 0.5) / 0.5
                    X[i] = img_array
                    
                    load_successful = True
    
                except Exception as e:
                    print(f"Error loading {img_path_to_process}. Replacing with a random file. Original file: {original_img_path}. Error: {e}")
                    img_path_to_process = random.choice(batch_files)
    
            if not load_successful:
                 print(f"CRITICAL: Failed to load a valid image for batch slot {i} after {max_attempts} attempts.")
                 X[i] = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3))
                 
        return X
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.file_list)
            np.random.shuffle(self.indexes)


def build_generator():
    '''Improved generator with better architecture for 128x128 RGB output'''

    noise_input = layers.Input(shape=(LATENT_DIM,), name='noise_input')

    x = layers.Dense(4 * 4 * 1024, use_bias=False)(noise_input)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)
    x = layers.Reshape((4, 4, 1024))(x)

    x = layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)

    generated_image = layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', 
                                           activation='tanh', use_bias=False, name='generated_image')(x)

    generator = keras.Model(inputs=noise_input, outputs=generated_image, name='Generator')
    return generator

def build_discriminator():
    '''Improved discriminator with dropout for 128x128 RGB input'''

    image_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name='image_input')

    x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(image_input)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(256, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(512, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(1024, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    validity = layers.Dense(1, activation='sigmoid', name='validity')(x)

    discriminator = keras.Model(inputs=image_input, outputs=validity, name='Discriminator')
    return discriminator


def find_latest_checkpoint(checkpoint_dir):
    '''Find the latest checkpoint epoch number'''
    if not os.path.exists(checkpoint_dir):
        return None, None
    
    generator_files = glob.glob(os.path.join(checkpoint_dir, 'generator2_epoch_*.keras'))
    discriminator_files = glob.glob(os.path.join(checkpoint_dir, 'discriminator2_epoch_*.keras'))
    
    if not generator_files or not discriminator_files:
        return None, None
    
    # Extract epoch numbers from filenames
    gen_epochs = []
    for f in generator_files:
        match = re.search(r'generator2_epoch_(\d+)\.keras', f)
        if match:
            gen_epochs.append(int(match.group(1)))
    
    disc_epochs = []
    for f in discriminator_files:
        match = re.search(r'discriminator2_epoch_(\d+)\.keras', f)
        if match:
            disc_epochs.append(int(match.group(1)))
    
    if not gen_epochs or not disc_epochs:
        return None, None
    
    # Find the latest epoch that has both generator and discriminator
    common_epochs = set(gen_epochs) & set(disc_epochs)
    if not common_epochs:
        return None, None
    
    latest_epoch = max(common_epochs)
    gen_path = os.path.join(checkpoint_dir, f'generator2_epoch_{latest_epoch:03d}.keras')
    disc_path = os.path.join(checkpoint_dir, f'discriminator2_epoch_{latest_epoch:03d}.keras')
    
    return latest_epoch, (gen_path, disc_path)


def find_latest_saved_image_epoch(result_dir):
    '''Find the latest epoch number from saved images'''
    if not os.path.exists(result_dir):
        return 0
    
    # Check both generated and comparison images
    generated_files = glob.glob(os.path.join(result_dir, 'new_generated_samples_vivian_epoch_*.png'))
    comparison_files = glob.glob(os.path.join(result_dir, 'comparison_vivian_epoch_*.png'))
    
    all_files = generated_files + comparison_files
    
    if not all_files:
        return 0
    
    epochs = []
    for f in all_files:
        # Match both numeric and "final" epochs
        match = re.search(r'epoch_(\d+)\.png', f)
        if match:
            epochs.append(int(match.group(1)))
    
    return max(epochs) if epochs else 0


class ImprovedDCGAN:
    '''Improved DCGAN with better training dynamics and checkpoint resumption'''
    
    def __init__(self, latent_dim=100, checkpoint_paths=None):
        self.latent_dim = latent_dim
        
        # Build or load networks
        if checkpoint_paths:
            print(f"Loading generator from: {checkpoint_paths[0]}")
            print(f"Loading discriminator from: {checkpoint_paths[1]}")
            self.generator = keras.models.load_model(checkpoint_paths[0])
            self.discriminator = keras.models.load_model(checkpoint_paths[1])
            print("Models loaded successfully!")
        else:
            print("Building new models from scratch...")
            self.generator = build_generator()
            self.discriminator = build_discriminator()
        
        # Different learning rates for G and D
        self.gen_optimizer = Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.999)
        self.disc_optimizer = Adam(learning_rate=0.0004, beta_1=0.5, beta_2=0.999)
        
        # Loss function and metrics
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.1)
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")
    
    def train_step(self, real_images):
        """Improved training step with better balance"""
        
        real_images = tf.convert_to_tensor(real_images, dtype=tf.float32)
        batch_size = tf.shape(real_images)[0]
        
        # Train discriminator
        with tf.GradientTape() as disc_tape:
            real_validity = self.discriminator(real_images, training=True)
            real_labels = tf.ones_like(real_validity) * 0.9
            real_loss = self.loss_fn(real_labels, real_validity)
            
            noise = tf.random.normal([batch_size, self.latent_dim])
            generated_images = self.generator(noise, training=True)
            fake_validity = self.discriminator(generated_images, training=True)
            fake_labels = tf.zeros_like(fake_validity) + 0.1
            fake_loss = self.loss_fn(fake_labels, fake_validity)
            
            disc_loss = (real_loss + fake_loss) / 2
            
        disc_gradients = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
        self.disc_optimizer.apply_gradients(zip(disc_gradients, self.discriminator.trainable_variables))
        
        # Train generator
        with tf.GradientTape() as gen_tape:
            noise = tf.random.normal([batch_size, self.latent_dim])
            generated_images = self.generator(noise, training=True)
            validity = self.discriminator(generated_images, training=True)
            
            gen_loss = self.loss_fn(tf.ones_like(validity), validity)
            
        gen_gradients = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        self.gen_optimizer.apply_gradients(zip(gen_gradients, self.generator.trainable_variables))
        
        # Update metrics
        self.gen_loss_tracker.update_state(gen_loss)
        self.disc_loss_tracker.update_state(disc_loss)
        
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }
    
    def generate_images(self, num_samples=1):
        '''Generate images'''
        noise = tf.random.normal([num_samples, self.latent_dim])
        generated_images = self.generator(noise, training=False)
        
        generated_images = (generated_images + 1) / 2.0
        
        return generated_images.numpy()
    
    def save_sample_images(self, epoch, num_samples=8):
        '''Save sample generated images'''
        
        generated_images = self.generate_images(num_samples)
        
        cols = min(4, num_samples)
        rows = (num_samples + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
        
        if rows == 1 and cols == 1:
            axes = [axes]
        elif rows == 1 or cols == 1:
            axes = axes.flatten() if hasattr(axes, 'flatten') else [axes]
        else:
            axes = axes.flatten()
        
        for i in range(num_samples):
            axes[i].imshow(generated_images[i])
            axes[i].set_title(f'Generated Vivian')
            axes[i].axis('off')
        
        for i in range(num_samples, len(axes)):
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.savefig(f'{RESULT_SAVE_DIR}/new_generated_samples_vivian_epoch_{epoch}.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"Saved sample images for epoch {epoch}")

    def save_comparison_images(self, epoch, data_generator, num_samples=8):
        '''Save comparison between real and generated images'''
        
        batch_idx = random.randint(0, len(data_generator) - 1)
        real_batch = data_generator[batch_idx]
        real_images = real_batch[:num_samples]
        
        generated_images = self.generate_images(num_samples)
        
        fig, axes = plt.subplots(2, num_samples, figsize=(num_samples * 2, 4))
        
        for i in range(num_samples):
            real_img = (real_images[i] + 1) / 2.0
            axes[0, i].imshow(real_img)
            axes[0, i].set_title(f'Real Vivian' if i == 0 else '')
            axes[0, i].axis('off')
            
            axes[1, i].imshow(generated_images[i])
            axes[1, i].set_title(f'Generated Vivian' if i == 0 else '')
            axes[1, i].axis('off')
        
        plt.tight_layout()
        plt.savefig(f'{RESULT_SAVE_DIR}/comparison_vivian_epoch_{epoch}.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"Saved comparison images for epoch {epoch}")


# %%
def train_vivian():
    '''Improved training function with checkpoint resumption'''
    
    print(f"\n{'='*60}")
    print(f"Training Vivian DCGAN with Checkpoint Resumption")
    print(f"{'='*60}")
    
    # Create necessary directories
    checkpoint_dir = 'checkpoints_128x128/vivian'
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(RESULT_SAVE_DIR, exist_ok=True)
    
    # Check for existing checkpoints
    latest_checkpoint_epoch, checkpoint_paths = find_latest_checkpoint(checkpoint_dir)
    start_epoch = 0
    
    if latest_checkpoint_epoch is not None:
        print(f"\n{'='*60}")
        print(f"CHECKPOINT FOUND: Epoch {latest_checkpoint_epoch}")
        print(f"{'='*60}")
        dcgan = ImprovedDCGAN(LATENT_DIM, checkpoint_paths=checkpoint_paths)
        start_epoch = latest_checkpoint_epoch
        print(f"Resuming training from epoch {start_epoch + 1}")
    else:
        print("No checkpoint found. Starting training from scratch.")
        dcgan = ImprovedDCGAN(LATENT_DIM)
    
    # Initialize data generator
    data_generator = SingleClassDataGenerator(
        dataset_path=DATASET_PATH,
        batch_size=BATCH_SIZE,
        max_images=MAX_IMAGES_PER_CLASS,
        shuffle=True
    )
    
    if len(data_generator.file_list) == 0:
        print(f"No images found in {DATASET_PATH}, stopping...")
        return
    
    print(f"Training samples: {len(data_generator.file_list)}")
    print(f"Batches per epoch: {len(data_generator)}")
    print(f"Training from epoch {start_epoch + 1} to {EPOCHS_PER_CLASS}")
    
    # Track loss history
    g_losses = []
    d_losses = []
    
    # Training loop - start from the next epoch after checkpoint
    for epoch in range(start_epoch, EPOCHS_PER_CLASS):
        actual_epoch_num = epoch + 1  # 1-indexed for display
        print(f"\nEpoch {actual_epoch_num}/{EPOCHS_PER_CLASS}")
        
        # Reset metrics
        dcgan.gen_loss_tracker.reset_state()
        dcgan.disc_loss_tracker.reset_state()
        
        # Training step for each batch
        progress_bar = tqdm(range(len(data_generator)), desc=f"Epoch {actual_epoch_num}")
        for batch_idx in progress_bar:
            real_images = data_generator[batch_idx]
            losses = dcgan.train_step(real_images)
            
            progress_bar.set_postfix({
                'G_Loss': f"{losses['g_loss']:.4f}",
                'D_Loss': f"{losses['d_loss']:.4f}",
            })
        
        # Record losses
        avg_g_loss = dcgan.gen_loss_tracker.result().numpy()
        avg_d_loss = dcgan.disc_loss_tracker.result().numpy()
        g_losses.append(avg_g_loss)
        d_losses.append(avg_d_loss)
        
        print(f"Epoch {actual_epoch_num} completed - "
              f"Avg G Loss: {avg_g_loss:.4f}, Avg D Loss: {avg_d_loss:.4f}")
        
        # Save sample images every 5 epochs
        if actual_epoch_num % 5 == 0 or epoch == start_epoch:
            dcgan.save_sample_images(actual_epoch_num)

        # Save comparison images every 10 epochs
        if actual_epoch_num % 10 == 0:
            dcgan.save_comparison_images(actual_epoch_num, data_generator)
        
        # Save model checkpoints every 30 epochs
        if actual_epoch_num % 30 == 0:
            dcgan.generator.save(f'{checkpoint_dir}/generator2_epoch_{actual_epoch_num:03d}.keras')
            dcgan.discriminator.save(f'{checkpoint_dir}/discriminator2_epoch_{actual_epoch_num:03d}.keras')
            print(f"Models saved at epoch {actual_epoch_num}")
        
        # Early stopping conditions
        if avg_d_loss < 0.01:
            print("Discriminator loss too low, stopping training to prevent mode collapse")
            break
            
        if avg_g_loss > 6.0:
            print("Generator loss too high, stopping training")
            break
            
        if len(g_losses) > 5:
            recent_g_avg = np.mean(g_losses[-5:])
            if avg_g_loss > recent_g_avg * 2:
                print("Generator loss spiking, stopping training")
                break
        
        # Shuffle data
        data_generator.on_epoch_end()
    
    # Save final models
    dcgan.generator.save('vivianxsp2GenNet.keras')
    dcgan.discriminator.save(f'{checkpoint_dir}/final_discriminator.keras')
    print(f"Final generator saved as vivianxsp2GenNet.keras!")
    
    # Generate final sample images
    dcgan.save_sample_images("final", 9)
    
    return dcgan


# %%
def main():
    '''Main training function with checkpoint resumption'''
    
    if not os.path.exists(DATASET_PATH):
        print("ERROR: Dataset directory not found!")
        print("Please ensure Vivian images are available at:", DATASET_PATH)
        return
    
    print("Starting Vivian DCGAN training with checkpoint resumption...")
    
    try:
        dcgan = train_vivian()
        print("\nTraining completed successfully!")
        print(f"Generator saved as: vivianxsp2GenNet.keras")
            
    except KeyboardInterrupt:
        print("\n\nTraining interrupted by user")
        print("Progress has been saved. Run the script again to resume training.")
    except Exception as e:
        print(f"\nError during training: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

# %%

2025-10-25 22:06:32.787670: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-25 22:06:35.792003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761397596.837667     490 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761397597.175596     490 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-25 22:06:39.688120: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Starting Vivian DCGAN training with checkpoint resumption...

Training Vivian DCGAN with Checkpoint Resumption

CHECKPOINT FOUND: Epoch 180
Loading generator from: checkpoints_128x128/vivian/generator2_epoch_180.keras
Loading discriminator from: checkpoints_128x128/vivian/discriminator2_epoch_180.keras


I0000 00:00:1761397663.669853     490 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3620 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Models loaded successfully!
Resuming training from epoch 181
Loading images from: vivian_nsfw_sfw_cropped_combined_faces
Found 5836 images
Training samples: 5836
Batches per epoch: 729
Training from epoch 181 to 400

Epoch 181/400


I0000 00:00:1761397668.738848     490 cuda_dnn.cc:529] Loaded cuDNN version 90300

poch 181: 100%|████████████████████████████████████████| 729/729 [07:52<00:00,  1.54it/s, G_Loss=1.9225, D_Loss=0.4102]

...


poch 371: 100%|████████████████████████████████████████| 729/729 [06:45<00:00,  1.80it/s, G_Loss=1.8800, D_Loss=0.4065]

Epoch 371 completed - Avg G Loss: 1.8800, Avg D Loss: 0.4065

Epoch 372/400



poch 372: 100%|████████████████████████████████████████| 729/729 [06:27<00:00,  1.88it/s, G_Loss=1.8843, D_Loss=0.4061]

Epoch 372 completed - Avg G Loss: 1.8843, Avg D Loss: 0.4061

Epoch 373/400


Epoch 373:   6%|██▎                                      | 41/729 [00:22<06:05,  1.88it/s, G_Loss=1.8800, D_Loss=0.4059]